Import Libraries

In [28]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd

Get all the property addresses

In [2]:
# rightmove main URL
### initialise search ### 
URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=" + str(i) + "&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


10  Pages available
Number of links acquired:  233
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/107409182#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/109937678#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/110514809#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/111645674#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114573920#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114832574#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115021484#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115645691#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106038#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106077#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116108690#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116233709#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/propertie

Fishing Properties

In [31]:
i = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")
    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    noOfBeds = soup.select('p')[1].text
    noOfBathrooms = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', cleanIds[i])
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            pass
    
    data = [str(price), str(propertyAddress), str(noOfBeds), str(noOfBathrooms), str(foundFloorPlanImage), str(mapLocation)]
    all_info = np.append(all_info, data)
    i = i+1 
    end = time()
    print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!')
    

all_info = np.split(all_info, len(cleanLinks))
print(all_info)

1 out of 233 It took 0.61 seconds!
2 out of 233 It took 0.6 seconds!
3 out of 233 It took 1.64 seconds!
4 out of 233 It took 0.84 seconds!
5 out of 233 It took 0.63 seconds!
6 out of 233 It took 0.56 seconds!
7 out of 233 It took 2.84 seconds!
8 out of 233 It took 0.58 seconds!
9 out of 233 It took 0.58 seconds!
10 out of 233 It took 0.62 seconds!
11 out of 233 It took 0.6 seconds!
12 out of 233 It took 0.8 seconds!
13 out of 233 It took 0.6 seconds!
14 out of 233 It took 1.64 seconds!
15 out of 233 It took 0.56 seconds!
16 out of 233 It took 0.56 seconds!
17 out of 233 It took 0.55 seconds!
18 out of 233 It took 0.57 seconds!
19 out of 233 It took 0.54 seconds!
20 out of 233 It took 0.82 seconds!
21 out of 233 It took 0.61 seconds!
22 out of 233 It took 1.66 seconds!
23 out of 233 It took 1.82 seconds!
24 out of 233 It took 0.59 seconds!
25 out of 233 It took 0.6 seconds!
26 out of 233 It took 0.6 seconds!
27 out of 233 It took 0.57 seconds!
28 out of 233 It took 0.82 seconds!
29 out 

Image Processing

In [33]:
for info in range(len(all_info)):
    print(str(info)+' out of '+str(len(all_info)))
    # print(all_info[info][4])
    
    # Load the img
    req = urllib.request.urlopen(all_info[info][4])
    arr = np.asarray(bytearray(req.read()))
    img = cv2.imdecode(arr, -1) # 'Load it as it is'

    # Display loaded image
    # cv2.imshow('Floor Plan', img)
    # if cv2.waitKey() & 0xff == 27: quit()

    # OCR
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    configs = r'--psm 6'
    try:
        rawText = pytesseract.image_to_string(img, config=configs)
    except:
        continue
    rawText = rawText.lower()
    # print(rawText)
    if 'area' in rawText:
        findArea = rawText.index('area') 
        newText = rawText[findArea+4:]
        try:
            findft = newText.index('ft') 
            newText = newText[:findft+2]
        except:
            continue
            
        result = re.sub(r'[^0-9,^.]', ' ', str(newText))
        newText = np.char.strip(result, ' ')
        newText = np.char.strip(newText, '. ')
        newText = np.char.strip(newText, ' .')
        newText = np.char.strip(newText, '\r')
        newText = str(newText)
        newText = newText[np.char.rfind(newText, ' ')+1:]
        # print(newText)
        updated_info = np.append(all_info, newText)
    else:
        # print('No area found...')
        updated_info = np.append(all_info, 'N/A')
        pass

print(updated_info)
    
    

0 out of 233
1 out of 233
2 out of 233
3 out of 233
4 out of 233
5 out of 233
6 out of 233
7 out of 233
8 out of 233
9 out of 233
10 out of 233
11 out of 233
12 out of 233
13 out of 233
No area found...
14 out of 233
No area found...
15 out of 233
16 out of 233
17 out of 233
18 out of 233
19 out of 233
No area found...
20 out of 233
21 out of 233
22 out of 233
23 out of 233
24 out of 233
25 out of 233
26 out of 233
27 out of 233
28 out of 233
29 out of 233
30 out of 233
31 out of 233
32 out of 233
33 out of 233
34 out of 233
35 out of 233
36 out of 233
37 out of 233
38 out of 233
39 out of 233
40 out of 233
41 out of 233
42 out of 233
43 out of 233
No area found...
44 out of 233
No area found...
45 out of 233
46 out of 233
47 out of 233
48 out of 233
49 out of 233
50 out of 233
51 out of 233
No area found...
52 out of 233
53 out of 233
54 out of 233
55 out of 233
56 out of 233
57 out of 233
58 out of 233
59 out of 233
60 out of 233
No area found...
61 out of 233
No area found...
62 out

In [35]:
df = pd.DataFrame(updated_info, columns = ['test'])#['price', 'propertyAddress', 'noOfBeds', 'noOfBathrooms', 'foundFloorPlanImage', 'mapLocation','area in ft'])
print(df)

                                                   test
0                                              £475,000
1                     Echo Court, Telegraph Avenue, NW9
2                                             Apartment
3                                                    ×3
4     https://media.rightmove.co.uk/dir/12k/11432/96...
...                                                 ...
1394                                         Maisonette
1395                                                 ×2
1396  https://media.rightmove.co.uk/dir/12k/11432/96...
1397  https://www.rightmove.co.uk/properties/9660361...
1398                                                796

[1399 rows x 1 columns]
